Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from sklearn.linear_model import LogisticRegression

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_lbls, valid_lbls, test_lbls =train_labels[:], valid_labels[:], test_labels[:]
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

# Logistic classifier with l2 reg

In [ ]:
def train_logistic_regression(data, lbls):
    print("Training logistic regression model...")
    model = LogisticRegression(penalty='l2')
    model.fit(data, lbls)
    print("Training done.")
    return model

def predictions(model, data, lbls):
    print("Predicting...")
    correct = 0
    data_len = data.shape[0]
    for i in xrange(data_len):
        sample = data[i,:]
        sample = sample.reshape(1, -1)
        lbl = lbls[i]
        pred = model.predict(sample)
        if pred[0] == lbl:
            correct += 1
    print("Accuracy: %.4f%%" % (float(correct) / data_len,))
    
for model_size in (50, 100, 500, 1000, 5000):
    print("Model size: %d" % (model_size,))
    model = train_logistic_regression(train_dataset[:50], train_lbls[:50])
    
    print("Validation set")
    predictions(model, valid_dataset , valid_lbls )

    print("Test set")
    predictions(model, test_dataset , test_lbls )
    
    print("-" * 50)
    
print("Training on all samples ...")
model = train_logistic_regression(train_dataset, train_lbls)

print("Validation set")
predictions(model, valid_dataset , valid_lbls )

print("Test set")
predictions(model, test_dataset , test_lbls )

Model size: 50
Training logistic regression model...
Training done.
Validation set
Predicting...
Accuracy: 0.5458%
Test set
Predicting...
Accuracy: 0.6002%
--------------------------------------------------
Model size: 100
Training logistic regression model...
Training done.
Validation set
Predicting...
Accuracy: 0.5458%
Test set
Predicting...
Accuracy: 0.6002%
--------------------------------------------------
Model size: 500
Training logistic regression model...
Training done.
Validation set
Predicting...
Accuracy: 0.5458%
Test set
Predicting...
Accuracy: 0.6002%
--------------------------------------------------
Model size: 1000
Training logistic regression model...
Training done.
Validation set
Predicting...
Accuracy: 0.5458%
Test set
Predicting...
Accuracy: 0.6002%
--------------------------------------------------
Model size: 5000
Training logistic regression model...
Training done.
Validation set
Predicting...
Accuracy: 0.5458%
Test set
Predicting...
Accuracy: 0.6002%
----------

# Neural network classifier with l2 reg

In [5]:
batch_size = 128
beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, 1024]))
    biases = tf.Variable(tf.zeros([1024]))

    
    hidden = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights) + biases
    )
    
    out_layer = tf.Variable(
        tf.truncated_normal([1024, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(hidden, out_layer) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += tf.nn.l2_loss(weights) * beta
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(
                    tf_valid_dataset, weights) + biases),
            out_layer) + out_biases
    )
        
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(
                    tf_test_dataset, weights) + biases),
            out_layer) + out_biases
    )
#     test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 428.068054
Minibatch accuracy: 13.3%
Validation accuracy: 30.1%
Minibatch loss at step 500: 151.614197
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 95.645798
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 77.714333
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 58.610420
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 44.540710
Minibatch accuracy: 89.1%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 33.914333
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 3500: 26.270380
Minibatch accuracy: 88.3%
Validation accuracy: 85.7%
Minibatch loss at step 4000: 20.438555
Minibatch accuracy: 91.4%
Validation accuracy: 86.3%
Minibatch loss at step 4500: 16.159775
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Minibatch loss at step 5000: 12.632383
Minibatch accuracy: 88.3%
Valid

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [13]:
batch_size = 128
beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, 1024]))
    biases = tf.Variable(tf.zeros([1024]))

    
    hidden = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights) + biases
    )
    
    out_layer = tf.Variable(
        tf.truncated_normal([1024, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(hidden, out_layer) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += tf.nn.l2_loss(weights) * beta
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(
                    tf_valid_dataset, weights) + biases),
            out_layer) + out_biases
    )
        
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(
                    tf_test_dataset, weights) + biases),
            out_layer) + out_biases
    )
#     test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step % 10 * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 452.156189
Minibatch accuracy: 8.6%
Validation accuracy: 35.5%
Minibatch loss at step 500: 121.641541
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 94.649559
Minibatch accuracy: 99.2%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 73.542549
Minibatch accuracy: 99.2%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 58.002384
Minibatch accuracy: 99.2%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 44.904022
Minibatch accuracy: 99.2%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 34.839497
Minibatch accuracy: 99.2%
Validation accuracy: 78.9%
Minibatch loss at step 3500: 27.070915
Minibatch accuracy: 99.2%
Validation accuracy: 78.8%
Minibatch loss at step 4000: 21.058861
Minibatch accuracy: 99.2%
Validation accuracy: 78.7%
Minibatch loss at step 4500: 16.385862
Minibatch accuracy: 99.2%
Validation accuracy: 78.6%
Minibatch loss at step 5000: 12.760720
Minibatch accuracy: 99.2%
Valida

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 128
beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, 1024]))
    biases = tf.Variable(tf.zeros([1024]))

    
    hidden = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights) + biases
    )
    hidden = tf.nn.dropout(hidden, 0.5) * 2.0
    
    out_layer = tf.Variable(
        tf.truncated_normal([1024, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(hidden, out_layer) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += tf.nn.l2_loss(weights) * beta
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(
                    tf_valid_dataset, weights) + biases),
            out_layer) + out_biases
    )
        
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(
                    tf_test_dataset, weights) + biases),
            out_layer) + out_biases
    )
#     test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step % 10 * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 961.783630
Minibatch accuracy: 14.8%
Validation accuracy: 22.5%
Minibatch loss at step 500: 189.115906
Minibatch accuracy: 96.9%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 215.779663
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 149.778091
Minibatch accuracy: 98.4%
Validation accuracy: 77.4%
Minibatch loss at step 2000: 105.465256
Minibatch accuracy: 97.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 58.542114
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 51.179089
Minibatch accuracy: 98.4%
Validation accuracy: 80.4%
Minibatch loss at step 3500: 38.677212
Minibatch accuracy: 98.4%
Validation accuracy: 80.3%
Minibatch loss at step 4000: 96.669632
Minibatch accuracy: 96.1%
Validation accuracy: 79.4%
Minibatch loss at step 4500: 47.360592
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 5000: 26.752182
Minibatch accuracy: 98.4%
V

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [32]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, 1024], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([1024]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([1024, 300], stddev=np.sqrt(2.0 / (1024 * 1024))))
    biases2 = tf.Variable(tf.zeros([300]))
    
    weights3 = tf.Variable(
        tf.truncated_normal([300, 50], stddev=np.sqrt(2.0 / (300 * 300))))
    biases3 = tf.Variable(tf.zeros([50]))
    
    out_layer = tf.Variable(
        tf.truncated_normal([50, num_labels], stddev=np.sqrt(2.0/ (50 * 50))))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden1 = tf.nn.dropout(hidden1, 0.8) * 1.25
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    hidden2 = tf.nn.dropout(hidden2, 0.5) * 2.0
    hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)
    hidden3 = tf.nn.dropout(hidden3, 0.8) * 1.25
    
    
    # Training computation.
    logits = tf.matmul(hidden3, out_layer) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) +
            tf.nn.l2_loss(weights3)) * beta
    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 4000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    hidden1v = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    hidden2v = tf.nn.relu(tf.matmul(hidden1v, weights2) + biases2)
    hidden3v = tf.nn.relu(tf.matmul(hidden2v, weights3) + biases3)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden3v, out_layer) + out_biases)
        
    hidden1t = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    hidden2t = tf.nn.relu(tf.matmul(hidden1t, weights2) + biases2)
    hidden3t = tf.nn.relu(tf.matmul(hidden2t, weights3) + biases3)
    test_prediction = tf.nn.softmax(tf.matmul(hidden3t, out_layer) + out_biases)
#     test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 700001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.699068
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 10000: 0.651006
Minibatch accuracy: 85.2%
Validation accuracy: 89.6%
Minibatch loss at step 20000: 0.399845
Minibatch accuracy: 92.2%
Validation accuracy: 90.8%
Minibatch loss at step 30000: 0.256545
Minibatch accuracy: 93.8%
Validation accuracy: 91.4%
Minibatch loss at step 40000: 0.233781
Minibatch accuracy: 95.3%
Validation accuracy: 91.5%
Minibatch loss at step 50000: 0.127255
Minibatch accuracy: 98.4%
Validation accuracy: 91.4%
Minibatch loss at step 60000: 0.233440
Minibatch accuracy: 93.8%
Validation accuracy: 91.5%
Minibatch loss at step 70000: 0.202936
Minibatch accuracy: 96.1%
Validation accuracy: 91.5%
Minibatch loss at step 80000: 0.176214
Minibatch accuracy: 96.9%
Validation accuracy: 91.5%
Minibatch loss at step 90000: 0.287351
Minibatch accuracy: 92.2%
Validation accuracy: 91.5%
Minibatch loss at step 100000: 0.179072
Minibatch accuracy: 96.1%
Validat